In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin, urlparse, parse_qs

In [2]:
# 수집해야할 날짜의 범위
date_range = (pd.date_range(start='20200501', end='20200606')).strftime("%Y%m%d").tolist()

base_url= 'https://news.naver.com/main/ranking/popularDay.nhn'
params = {
    'rankingType' : 'popular_day',
    'date' : ''
}
ranking_news_list = []
for date in date_range:
    params['date'] = date
    print('date', date)

    resp = requests.get(base_url,params=params)
    soup = BeautifulSoup(resp.text)
    ranking_wrap = soup.select('div.ranking_section li dt a')
    
    date_news_dict = {}
    news_list = []
    
    for ranking in ranking_wrap[:-15]:
        news_list.append(urljoin(base_url,ranking.get('href')))

    date_news_dict[date] = news_list
    
    ranking_news_list.append(date_news_dict)


date 20200501
date 20200502


C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


date 20200503
date 20200504
date 20200505
date 20200506
date 20200507
date 20200508
date 20200509
date 20200510
date 20200511
date 20200512
date 20200513
date 20200514
date 20200515
date 20200516
date 20200517
date 20200518
date 20200519
date 20200520
date 20200521
date 20200522
date 20200523
date 20200524
date 20200525
date 20200526
date 20200527
date 20200528
date 20200529
date 20200530
date 20200531
date 20200601
date 20200602
date 20200603
date 20200604
date 20200605
date 20200606


In [3]:
ranking_news_list[0]

{'20200501': ['https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=005&aid=0001317046&date=20200501&type=1&rankingSectionId=100&rankingSeq=1',
  'https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=023&aid=0003527895&date=20200501&type=1&rankingSectionId=100&rankingSeq=2',
  'https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=055&aid=0000811740&date=20200501&type=2&rankingSectionId=100&rankingSeq=3',
  'https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=025&aid=0002997511&date=20200501&type=1&rankingSectionId=100&rankingSeq=4',
  'https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=001&aid=0011583797&date=20200501&type=1&rankingSectionId=100&rankingSeq=5',
  'https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=055&aid=0000811630&date=20200501&type=2&rankingSec

In [4]:
finally_news_list = []


for ranking_list in ranking_news_list:
    for yyyymmdd, ranking in ranking_list.items():
        for ranking_news in ranking:

            news_dict = {}
            resp_news = requests.get(ranking_news)
            soup_news = BeautifulSoup(resp_news.text)

            # 기사 제목
            header_wrap = soup_news.select('div.article_info h3')
            if header_wrap:
                title = header_wrap[0].text
            else:
                 continue    

            # 섹션
            section = soup_news.find('em', class_='guide_categorization_item')

            # 기사 ID
            parsed_url = urlparse(ranking_news)
            qs = parse_qs(parsed_url.query)
            oid = qs['oid'][0]
            aid = qs['aid'][0]
            print(oid, aid)

            # 내용
            summary_url = 'https://tts.news.naver.com/article/{}/{}/summary'.format(oid, aid)
            resp_summary = requests.get(summary_url)
            soup_summary = BeautifulSoup(resp_summary.text)
            summary = soup_summary.text.split(':')[-1].replace('}','')


            news_dict = {
                '날짜' : yyyymmdd[:4]+'-'+yyyymmdd[4:6]+'-'+yyyymmdd[6:],
                'title' : title,
                'category' : section.text,
                'summary' : summary.replace('"','')
            }

            # 이미지 
            image_wrap = soup_news.find('span', class_="end_photo_org")

            if image_wrap:
                news_dict['img'] = image_wrap.find('img').get('src')
            finally_news_list.append(news_dict)


005 0001317046
023 0003527895
055 0000811740
025 0002997511
001 0011583797
055 0000811630
025 0002997407
009 0004567254
016 0001668798
008 0004402866
025 0002997394
005 0001317043
025 0002997503
023 0003527973
437 0000237154
025 0002997580
025 0002997602
008 0004403075
023 0003528146
001 0011584331
008 0004402866
437 0000237217
008 0004402941
025 0002997576
008 0004402981
025 0002997622
001 0011584665
023 0003528144
052 0001435240
003 0009843753
001 0011585201
023 0003528237
023 0003528189
003 0009844215
001 0011585601
629 0000024629
015 0004333837
008 0004403093
025 0002997667
015 0004334026
005 0001317275
025 0002997665
023 0003528245
022 0003461713
022 0003461681
028 0002495919
023 0003528256
025 0002997854
006 0000102466
014 0004418768
025 0002997852
009 0004568298
023 0003528365
025 0002997864
015 0004334404
025 0002997905
014 0004418534
001 0011587135
023 0003528310
023 0003528414
025 0002998224
022 0003462363
008 0004403976
008 0004403787
015 0004334930
009 0004568799
015 000433

025 0003006958
008 0004420906
008 0004420808
437 0000239858
448 0000299170
025 0003007011
025 0003007002
008 0004420910


In [5]:
finally_news_list[0]

{'날짜': '2020-05-01',
 'title': '지성호 “김정은, 지난 주말 수술 쇼크로 사망했다”',
 'category': '정치',
 'summary': '탈북민 출신 인권운동가 지성호 미래한국당 국회의원 당선인이 김정은 북한 국무위원장의 사망설을 강하게 제기하며 후임으로 김 위원장의 여동생인 김여정이 유력하다는 관측을 내놨다.김 위원장이 20일 가까이 공식 석상에 모습을 드러내지 않자 건강이상설을 넘어 사망설까지 나오는 상황이어서 주목된다.',
 'img': 'https://imgnews.pstatic.net/image/005/2020/05/01/611116110014535978_1_20200501113603896.jpg?type=w647'}

In [6]:
df = pd.DataFrame(finally_news_list)
df.to_csv('naver_news_crawling.csv', encoding='cp949')

In [7]:
df

,날짜,title,category,summary,img
0,2020-05-01,"지성호 “김정은, 지난 주말 수술 쇼크로 사망했다”",정치,탈북민 출신 인권운동가 지성호 미래한국당 국회의원 당선인이 김정은 북한 국무위원장의...,https://imgnews.pstatic.net/image/005/2020/05/...
1,2020-05-01,"지성호 ""김정은 지난 주말 사망.. 99% 확신"" 주장",정치,탈북자 출신 인권운동가 지성호 미래한국당 비례대표 당선자는 1일 \김정은이 사망한 ...,https://imgnews.pstatic.net/image/023/2020/05/...
2,2020-05-01,"[인터뷰] ""김정은 사망 99% 확신"" 지성호 주장 이유는?",정치,"오늘 우리 지성호 당선인, 또 미래통합당의 태구민, 태영호 당선인이 대한민국 국회...",https://imgnews.pstatic.net/image/055/2020/05/...
3,2020-05-01,"휴가 취소하고 걸어서 곰탕집 간 文 ""식당 이용 활발했으면""",정치,문재인 대통령이 1일 참모진들과 청와대 주변 한 곰탕집을 찾아 오찬을 했다.문 대통...,https://imgnews.pstatic.net/image/025/2020/05/...
4,2020-05-01,"지성호 ""김정은 사망 99% 확신"", 청와대 ""특이동향 없다""(종합)",정치,탈북자 출신으로 4·15 총선에서 미래한국당 비례대표로 당선된 지성호 당선인은 1일...,https://imgnews.pstatic.net/image/001/2020/05/...
...,...,...,...,...,...
550,2020-06-06,"이용수 할머니 ""억울…윤미향, 죄 지었으면 벌 받아야""",사회,오늘 대구에선 일본군 '위안부' 피해자를 추모하는 행사가 열렸습니다.이 할머니가 외...,NaN
551,2020-06-06,양천구 탁구장 관련 17명 확진…수도권 집단감염 '비상’,사회,탁구장 관련 확진자도 17명으로 늘었습니다.이 남성은 지난달 28일부터 지난 3일까...,NaN
552,2020-06-06,"""조국, 여배우 밀어줬다"" 주장한 김용호 前 기자 검찰 송치",사회,경찰이 6일 유튜브 방송에서 조국 전 법무부 장관이 한 여배우를 후원했다고 주장한 ...,https://imgnews.pstatic.net/image/025/2020/06/...
553,2020-06-06,[애니띵]겨우 사흘 살고 떠났다···美 울린 '두 얼굴의 고양이',사회,"남들과 다르지만 그래서 더 특별한 고양이, 비스킷과 그래비를 소개합니다.하지만, 비...",https://imgnews.pstatic.net/image/025/2020/06/...
